# Original Jeepney Routes - Fitness Function

## Route Connection and Genetic Algorithm Functions

### Setting Up

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import networkx as nx
import shapely
import folium
import geojson
import math
import osmnx as ox
from rtree import index as rtree_index
import pickle
import copy
import utm
import csv

from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, LineString, Point
from geopy.distance import geodesic
from shapely.ops import split

from __future__ import absolute_import, division
from math import radians, sin, cos, sqrt, atan2, exp, log
import webbrowser
import random
from scipy.spatial import KDTree
from scipy.spatial.distance import euclidean

ox.settings.log_console=True
ox.settings.use_cache=True

In [67]:
# Defining classes for the dataframes      
class stopCandidate:
    def __init__(self, lat, long, isTranspo, id):
        self.lat = lat
        self.long = long
        self.isTranspo = isTranspo
        self.id = id #Stop ID
        
    def getLat(self):
        return self.lat
    
    def getLong(self):
        return self.long
    
    def getDegree(self):
        return self.degree
    
class networkObj:
    def __init__(self, route_network, graph):
        self.fitness_score = 0
        self.graph = graph
        self.routes = route_network
        
class Route:
    def __init__(self, route, route_id, length):
        self.route = route
        self.route_id = route_id
        self.length = length

In [68]:
# For units
def degrees_to_meters(angle_degrees):
    return angle_degrees * 6371000 * math.pi / 180

def meters_to_degrees(distance_meters):
    return distance_meters / 6371000 * 180 / math.pi

In [69]:
# Import and Export networks or graphs to pickle
def export_networks(networks, path):
    with open(path, 'wb') as f:
        pickle.dump(networks, f)

def import_networks(path):
    with open(path, 'rb') as f:
        routes = pickle.load(f)
    return routes

In [70]:
import re

def process_csv(file_path):
    with open(file_path, mode='r', newline='') as file:
        point_pattern = re.compile(r'POINT \(([-\d.]+) ([-\d.]+)\)')
        reader = csv.DictReader(file)
        
        # Variables to hold the current route list and all routes list
        current_route_number = None
        current_route_list = []
        all_routes_list = []

        for row in reader:
            wkt = row['WKT']
            route_number = row['Route Number']
            
            match = point_pattern.match(wkt)
            if match:
                x, y = match.groups()
                x, y = float(x), float(y)
            else:
                x, y = None, None

            # Check if the route number has changed
            if route_number != current_route_number:
                # If there's a current route list, append it to the all routes list
                if current_route_list:
                    all_routes_list.append(current_route_list)
                
                # Start a new route list
                current_route_list = []
                current_route_number = route_number

            # Append the current row to the current route list
            current_route_list.append([(x, y), route_number])

        # Append the last route list to the all routes list
        if current_route_list:
            all_routes_list.append(current_route_list)
        
    return all_routes_list

#### Graph and Features

In [71]:
# GENERATION OF MAIN CITY GRAPH
# IF FIRST TIME RUNNING, RUN THIS CODE TO GENERATE THE GRAPH
def generate_graph():
    mode = 'drive'
    graph = ox.graph_from_place(select_city, network_type = mode) # Generate graph of Metro manila
    ox.save_graphml(graph, city_file) # Save it as a file

def load_graph():
    graph = ox.load_graphml(city_file)
    
    print("Graph loaded successfully")
    print("NUMBER OF EDGES: ", graph.number_of_edges())
    print("NUMBER OF NODES: ", graph.number_of_nodes())
    print('\n')
    return graph

select_city = "Metro Manila, Philippines"
city_file = 'map/Metro Manila.graphml'
#generate_graph()
METROMANILA_GRAPH = load_graph()

select_city = "Manila, Philippines"
city_file = 'map/Manila.graphml'
#generate_graph()
MANILA_GRAPH = load_graph()

select_city = "Mandaluyong, Philippines"
city_file = 'map/Mandaluyong.graphml'
#generate_graph()
MANDALUYONG_GRAPH = load_graph()

select_city = "Makati, Philippines"
city_file = 'map/Makati.graphml'
#generate_graph()
MAKATI_GRAPH = load_graph()

Graph loaded successfully
NUMBER OF EDGES:  149172
NUMBER OF NODES:  59386


Graph loaded successfully
NUMBER OF EDGES:  12617
NUMBER OF NODES:  4926


Graph loaded successfully
NUMBER OF EDGES:  2374
NUMBER OF NODES:  989


Graph loaded successfully
NUMBER OF EDGES:  5505
NUMBER OF NODES:  2270




In [72]:
### For Filtering the roads and other features
# GETTING ROADS AND WATERWAYS

# Get all the roads in Manila
manila_road = ox.graph_to_gdfs(MANILA_GRAPH,nodes=False, edges=True)
filtered_roads = manila_road[manila_road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# Get all the roads with the allowed road types
filtered_roads_strings_manila = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 

# Total Distance of the allowed road types
TOTAL_NETWORK_ROAD_DISTANCE_MANILA = 0
for index, row in filtered_roads_strings_manila.iterrows():
    TOTAL_NETWORK_ROAD_DISTANCE_MANILA += row['length']
    
# MAKATI
makati_road = ox.graph_to_gdfs(MAKATI_GRAPH,nodes=False, edges=True)
filtered_roads = makati_road[makati_road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# Get all the roads with the allowed road types
filtered_roads_strings_makati = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 

# Total Distance of the allowed road types
TOTAL_NETWORK_ROAD_DISTANCE_MAKATI = 0
for index, row in filtered_roads_strings_makati.iterrows():
    TOTAL_NETWORK_ROAD_DISTANCE_MAKATI += row['length']


# MANDALUYONG
mandaluyong_road = ox.graph_to_gdfs(MANDALUYONG_GRAPH,nodes=False, edges=True)
filtered_roads = mandaluyong_road[mandaluyong_road['junction'].isna()]

# Separate roads whose highway types are only one value and those that are more than 1 (lists)
rows_with_lists = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, list))]
rows_with_strings = filtered_roads[filtered_roads['highway'].apply(lambda x: isinstance(x, str))]

# Allowed Roads to place stops
filter_options = ['primary', 'secondary', 'tertiary', 'trunk', 'unclassified']

# Get all the roads with the allowed road types
filtered_roads_strings_mandaluyong = rows_with_strings.loc[rows_with_strings['highway'].isin(filter_options)] 

# Total Distance of the allowed road types
TOTAL_NETWORK_ROAD_DISTANCE_MANDALUYONG = 0
for index, row in filtered_roads_strings_mandaluyong.iterrows():
    TOTAL_NETWORK_ROAD_DISTANCE_MANDALUYONG += row['length']

### Reading Data

In [234]:
# Reading all points

manila_points_gdf = gpd.read_file('./City Data/Manila_point.geojson')
mandaluyong_points_gdf = gpd.read_file('./City Data/Mandaluyong_point.geojson')
makati_points_gdf = gpd.read_file('./City Data/Makati_point.geojson')

manila_transpo_gdf = manila_points_gdf[manila_points_gdf['amenity'] == 'transportation']
mandaluyong_transpo_gdf = mandaluyong_points_gdf[mandaluyong_points_gdf['amenity'] == 'transportation']
makati_transpo_gdf = makati_points_gdf[makati_points_gdf['amenity'] == 'transportation']

In [235]:
# Index for optimization
manila_idx = rtree_index.Index()
for i, row in manila_transpo_gdf.iterrows():
    manila_idx.insert(i, row['geometry'].bounds)
    
mandaluyong_idx = rtree_index.Index()
for i, row in mandaluyong_transpo_gdf.iterrows():
    mandaluyong_idx.insert(i, row['geometry'].bounds)
    
makati_idx = rtree_index.Index()
for i, row in makati_transpo_gdf.iterrows():
    makati_idx.insert(i, row['geometry'].bounds)
    

In [236]:
# Function to Extract all the stop points per row
def extract_points(df):
    routes = []
    columns = list(df.columns)
    
    for index, row in df.iterrows():
        points = []
        for col in columns:
            cell_value = row[col]
            if pd.isna(cell_value):
                continue
                
            if cell_value.endswith(']'):
                cell_value = cell_value[:-1] + ',' # Remove the last bracket if it's there
                
            data = eval(cell_value)[0]
            
            lat = data[0]
            lon = data[1]
            points.append((lat, lon))
        routes.append(points)
    return routes

In [237]:
# Reading the original jeepney route data
manila_df = pd.read_csv('Original Jeepney Routes/Original Data/Manila Routes.csv')
makati_df = pd.read_csv('Original Jeepney Routes/Original Data/Makati Routes.csv')
mandaluyong_df = pd.read_csv('Original Jeepney Routes/Original Data/Mandaluyong Routes.csv')

manila_route_stops = extract_points(manila_df)
makati_route_stops = extract_points(makati_df)
mandaluyong_route_stops = extract_points(mandaluyong_df)

In [238]:
# REVISED ROUTES
file_path = 'Original Jeepney Routes/Original Data/Existing Jeepney Routes (Mandaluyong).csv'
mandaluyong_route_stops = process_csv(file_path)

### Stop Placement

In [209]:
# Checks if point is out of the specific geographic location
def remove_if_out_of_bounds(city_graph, routes, tolerance):
    nodes = ox.graph_to_gdfs(city_graph, edges=False)
    min_x, min_y, max_x, max_y = nodes.total_bounds
    
    new_routes = []
    for route in routes:
        new_route = []
        for point,name in route:
            long = point[0]
            lat = point[1]
            
            if min_x <= long <= max_x and min_y <= lat <= max_y:
                # Convert the graph to GeoDataFrame for easier spatial operations
                nearest_edge, distance_to_edge = ox.distance.nearest_edges(city_graph, X=long, Y=lat, return_dist=True)
                
            
                # Return True if any distance is less than the tolerance
                if distance_to_edge < tolerance:
                    #print(distance_to_edge)
                    new_route.append([point, name])
                else:
                    break
        new_routes.append(new_route)
            
    return new_routes

In [80]:
# This function checks if there are stops that are transpo stop and labels them as such
def get_close_points(idx, point, max_distance=50):
    close_points = [i for i in idx.intersection((point.x - max_distance, point.y - max_distance, point.x + max_distance, point.y + max_distance))]
    return close_points

# def check_transpo_stops(routes, point_gdf, city_idx, max_distance_to_stop):
#     transpo_points = []
#     non_transpo_points = []
#     new_route_list_with_stop_id = []
    
#     stop_id = 0
#     for route in routes:
#         new_route = []
#         for point in route:
#             lat = point[0]
#             long = point[1]
#             isTranspo = False
            
#             close_points = get_close_points(city_idx, Point(long, lat))
#             for i, row in point_gdf.loc[close_points].iterrows():
#                 distance_to_point = geodesic(point, (row['y'], row['x'])).meters
                
#                 # Check if distance is within 100 meters and it's a transportation amenity
#                 if distance_to_point <= max_distance_to_stop and row['amenity'] == 'transportation':
#                     isTranspo = True
#                     break
                
#             new_stop = stopCandidate(lat, long, isTranspo, stop_id)
#             new_route.append(new_stop)
            
#             if isTranspo:
#                 transpo_points.append(new_stop)
#             else:
#                 non_transpo_points.append(new_stop)
#             stop_id += 1
        
#         new_route_list_with_stop_id.append(new_route)
        
#     return transpo_points, non_transpo_points, new_route_list_with_stop_id
        

def check_transpo_stops(routes, point_gdf, city_idx, max_distance_to_stop):
    transpo_points = []
    non_transpo_points = []
    new_route_list_with_stop_id = []
    
    stop_id = 0
    for route in routes:
        new_route = []
        for point, name in route:
            long = point[0]
            lat = point[1]
            isTranspo = False
            
            close_points = get_close_points(city_idx, Point(long, lat))
            for i, row in point_gdf.loc[close_points].iterrows():
                distance_to_point = geodesic((lat, long), (row['y'], row['x'])).meters
                
                # Check if distance is within 100 meters and it's a transportation amenity
                if distance_to_point <= max_distance_to_stop and row['amenity'] == 'transportation':
                    isTranspo = True
                    break
                
            new_stop = stopCandidate(lat, long, isTranspo, stop_id)
            new_route.append(new_stop)
            
            if isTranspo:
                transpo_points.append(new_stop)
            else:
                non_transpo_points.append(new_stop)
            stop_id += 1
        
        new_route_list_with_stop_id.append(new_route)
        
    return transpo_points, non_transpo_points, new_route_list_with_stop_id

In [81]:
# x - lon - 120
# y - lat - 14
# LineString(lon, lat)
# geodesic(lat, lon)

def add_point_to_graph(point, graph_of_stops, list_of_stops, is_transpo):
    lat = point.lat
    long = point.long
    # Find the nearest edge to the location point
    nearest_edge = ox.distance.nearest_edges(graph_of_stops, X=long, Y=lat, return_dist=False)
    node1, node2, key = nearest_edge
    edge = graph_of_stops.get_edge_data(node1, node2)
    data = list(edge.values())[0]
        
    # get the line
    if 'geometry' not in data:
        gdf_edges = ox.graph_to_gdfs(graph_of_stops, nodes=False, edges=True)
        line = gdf_edges.loc[nearest_edge]['geometry']
    else:
        line = data['geometry']
        
    point_a = (graph_of_stops.nodes[node1]['y'], graph_of_stops.nodes[node1]['x'])
    point_c = (graph_of_stops.nodes[node2]['y'], graph_of_stops.nodes[node2]['x']) 
    

    graph_of_stops.add_node(point.id, x=long, y=lat, isTranspo=is_transpo)
    list_of_stops.append(point)

    # Get the points
    point_b = (lat, long) # y,x
    
    # Calculate the new distances
    distance_ab = geodesic(point_a, point_b).meters
    distance_bc = geodesic(point_b, point_c).meters
        
    # Get the edge data and adjust distances
    edge_data = data.copy()
    edge_data['length'] = distance_ab
    graph_of_stops.add_edge(node1, point.id, **edge_data)
    
    edge_data['length'] = distance_bc
    graph_of_stops.add_edge(point.id, node2, **edge_data)

    # Remove the original edge
    graph_of_stops.remove_edge(node1, node2)
        
def calculate_coordinate_along_edge(edge, position):
    # Calculate the coordinate along the edge at the given position
    point = edge.interpolate(position)
    return point.x, point.y

In [82]:
# CREATING STOPS
# It should return a list of coordinates/nodes for stop and a graph of stops
# if residential area, check if the population density

def place_stops_on_roads(transpo_points, non_transpo_points, graph_of_stops, list_of_stops):
    
    for point in transpo_points:
        add_point_to_graph(point, graph_of_stops, list_of_stops, True)
        
    for point in non_transpo_points:
        add_point_to_graph(point, graph_of_stops, list_of_stops, False)

In [83]:
def merge_stops(stops_list):
    stop_dict = {}
    # Sort the stops by latitude and then by longitude
    stops = sorted(stops_list, key=lambda x: (x.lat, x.long), reverse=True)
    

    grouped_stops = []
    current_group = [stops[0]]

    for i in range(1, len(stops)):
        distance = geodesic((current_group[-1].lat, current_group[-1].long), (stops[i].lat, stops[i].long)).meters
        if distance <= 20: # Distance between the points
            current_group.append(stops[i])
        else:
            grouped_stops.append(current_group)
            current_group = [stops[i]]

    # Last group
    if current_group:
        grouped_stops.append(current_group)

    
    for group in grouped_stops:
        
        if len(group) == 1:
            continue
        
        # Getting the main stop
        main_stop = None
        for stop in group:
            if stop.isTranspo:
                main_stop = stop
                break
        # If no transpo stop, let it be the first stop
        if main_stop == None:
            main_stop = group[0]
        
        for stop in group:
            if stop.id != main_stop.id:
                stop_dict[stop.id] = main_stop
        
    return stop_dict

In [84]:
def change_merged_stops(route_list, non_transpo_points_Manila, transpo_points_Manila, stop_dict):
    transpo_list = []
    non_transpo_list = []
    for route in route_list:
        for stop in route:
            if stop.id in stop_dict:
                index = route.index(stop)
                route[index] = stop_dict[stop.id]
    
    # Make sure there are no duplicates
    for i, route in enumerate(route_list):
        seen = set()
        unique_stops = []
        for stop in route:
            if stop.id not in seen:
                unique_stops.append(stop)
                seen.add(stop.id)
        route_list[i] = unique_stops
                
    for stop in non_transpo_points_Manila:
        if stop.id not in stop_dict:
            non_transpo_list.append(stop)
            
    for stop in transpo_points_Manila:
        if stop.id not in stop_dict:
            transpo_list.append(stop)
            
    return transpo_list, non_transpo_list

### Route Network Generation

In [85]:
def haversine(lat1, lon1, lat2, lon2):
    # Use geopy's geodesic function to calculate the distance
    distance = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    return distance

In [86]:
def generate_route_network(routes, graph_of_stops):
    overall_graph = nx.MultiDiGraph() # The route network graph
    route_network = []
    reverse_route_network = []
    num_routes = 0 # Count number of routes
    
    # Check first if there are paths
    for route in routes:
        prev_stop = None
        for stop in route:
            if prev_stop == None:
                prev_stop = stop
                continue
            
            if nx.has_path(graph_of_stops, prev_stop.id, stop.id):
                print(f"{prev_stop.id} - {stop.id}")
                prev_stop = stop
            else:
                route_index = routes.index(route)
                stop_index = route.index(prev_stop)
                routes[route_index] = route[:stop_index+1]
                break
        print()
    
    new_routes = []
    for route in routes:
        if len(route) > 3:
            new_routes.append(route)
    routes = new_routes
    
    print()
    print("Real route")
    for route in routes:
        route_id = f'{num_routes}-A' # This will be used as a key for the edge
        route_list = []
        prev_stop = None
        totalDistance = 0
        
        for stop in route: 
            if prev_stop == None:
                prev_stop = stop
                continue

            print(f"{prev_stop.id} - {stop.id}")
            
            path = nx.shortest_path(graph_of_stops, prev_stop.id, stop.id)
            route_list.append(path)
            
            distance_travelled = 0
            # Get the total distance from point A to point B
            for i in range(len(path)-1):
                node_data = graph_of_stops.nodes[path[i]]
                next_node_data = graph_of_stops.nodes[path[i+1]]
                distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
            
            overall_graph.add_node(prev_stop.id, lat=prev_stop.lat, lon=prev_stop.long, isTranspo=prev_stop.isTranspo) # The origin
            overall_graph.add_node(stop.id, lat=stop.lat, lon=stop.long, isTranspo=stop.isTranspo) # The dest
            overall_graph.add_edge(prev_stop.id, stop.id, key=route_id, road_path = path, distance = distance_travelled) # Add edge
            prev_stop = stop
            totalDistance += distance_travelled
            
            
        print()
        new_route_obj = Route(route_list, route_id, totalDistance)
        reverse_route = get_reverse_route(new_route_obj, graph_of_stops, overall_graph)
        route_network.append(new_route_obj)
        reverse_route_network.append(reverse_route)
        num_routes += 1
               
    return route_network, overall_graph, reverse_route_network
        
        

In [87]:
# Method 1
# Iterate through each node in reverse and get the shortest path

def get_reverse_route(route, graph_of_stops, overall_graph):
    paths = route.route
    reverse_paths = paths[::-1]
    reverse_route_list = []
    reverse_route_id = route.route_id[:-1] + 'B' # Reverse Route ID
    totalDistance = 0
    
    stop_list = []
    for connection in reverse_paths:
        reverse_route_path = connection[::-1]
        end_node = reverse_route_path[-1]
        
        # Getting the edges of the start node only for the first connection
        if reverse_paths.index(connection) == 0:
            start_node = reverse_route_path[0] # Start node is critical as it should be the same as the last stop previous connection (If this is not the first connection)
                    
        if not nx.has_path(graph_of_stops, start_node, end_node):
            return None
        else:
            stop_list.append([start_node, end_node])
                
        start_node = end_node # Next connection start node is the end node of the previous connection
        
    print(stop_list)
    for connection in stop_list:
        start_node = connection[0]
        end_node = connection[-1]
        
        reverse_route = nx.shortest_path(graph_of_stops, start_node, end_node, weight='length')
        reverse_route_list.append(reverse_route)
        
        distance_travelled = 0
        # Get the total distance from point A to point B
        for i in range(len(reverse_route)-1):
            node_data = graph_of_stops.nodes[reverse_route[i]]
            next_node_data = graph_of_stops.nodes[reverse_route[i+1]]
            distance_travelled += haversine(node_data['y'], node_data['x'], next_node_data['y'], next_node_data['x'])
        
        # Finally, Add the edge
        if reverse_route[0] not in overall_graph:
            overall_graph.add_node(reverse_route[0], **graph_of_stops.nodes[reverse_route[0]])
        if reverse_route[-1] not in overall_graph:
            overall_graph.add_node(reverse_route[-1], **graph_of_stops.nodes[reverse_route[-1]])
            
        overall_graph.add_edge(reverse_route[0], reverse_route[-1], key=reverse_route_id, road_path = reverse_route, distance = distance_travelled) # Add edge
        totalDistance += distance_travelled

    # print("REVERSE ROUTE DISTANCE: ", totalDistance)
    return Route(reverse_route_list, reverse_route_id, totalDistance)

### Fitness Function

In [88]:
# Fitness function

def compute_fitness_score(road_snapped_network_graph, num_failure_removal,
                          weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, total_road_network_distance, graph_of_stops):


    random_failure_robustness = compute_random_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_random_failure_robustness = weight_random_failure * random_failure_robustness

    targeted_failure_robustness = compute_targeted_failure_robustness(road_snapped_network_graph, num_failure_removal)
    weighted_targeted_failure_robustness = weight_targeted_failure * targeted_failure_robustness

    connectivity_score, transpo_stop_ratio,num_intersections, avg_transpo_degree = compute_connectivity(road_snapped_network_graph)
    weighted_connectivity = weight_connectivity * connectivity_score
    
    network_coverage = get_network_coverage(road_snapped_network_graph, total_road_network_distance, graph_of_stops)
    weighted_network_coverage = weight_network_coverage * network_coverage
    
    data = {
        'Random Failure Score': weighted_random_failure_robustness,
        'Target Failure Score': weighted_targeted_failure_robustness,
        'Connectivity': weighted_connectivity,
        'Transpo Stop Ratio': transpo_stop_ratio,
        'Num Intersections': num_intersections,
        'Average Transpo Degree': avg_transpo_degree,
        'Network Coverage': weighted_network_coverage
    }

    # Will use this return for now to utilize target and random failure nodes 
    return weighted_connectivity + weighted_network_coverage - weighted_random_failure_robustness - weighted_targeted_failure_robustness, data
    # return weighted_radius_of_gyration


### WRITTEN IN PSEUDOCODE
def compute_connectivity(network):
    # External connectivity - measure how connected is the jeepney route network with other modes of transpo
    
    # Get the ratio of transportation stops to total stops in the network
    transpo_stops = [node for node, node_data in network.nodes(data=True) if node_data['isTranspo'] == True]
    total_stops = len(network.nodes(data=True))
    transpo_stop_ratio = len(transpo_stops) / total_stops

    # Get the average degree of all transportation stops in the network
    if len(transpo_stops) > 0:
        avg_transpo_degree = sum(network.degree(stop) for stop in transpo_stops) / len(transpo_stops)
    else:
        avg_transpo_degree = 1

    # Find a way to normalize the two values and combine them 

    # Internal connectivity - measure how connected is each jeepney route to other jeepney routes
                    
    # This counts how many nodes have intersections (Meaning node is connected to more than one route by route ID)
    num_intersections = 0
    for node, node_data in network.nodes(data=True):
        unique_ids = set()  # Use a set to automatically handle uniqueness
        edges = network.edges(node, keys=True)  # Get all edges connected to the node with keys
        for u, v, key in edges:
            if isinstance(key, str) and '-' in key:
                id_str = key.split('-')[0]  # Get route id number
                unique_ids.add(id_str)  # Add to the set
        if len(unique_ids) > 1:
            num_intersections += 1

    
    # Change these weights based on what the expected values for 
    # the transpo_stop_ratio, avg_transpo_degree, and num_intersections will be
    external_weight = 0.5
    internal_weight = 0.5
    
    # TODO: Delete this
    # print("Transpo stop ratio: ", transpo_stop_ratio)
    # print("Num intersections: ", num_intersections)
    # print("Average degree: ", avg_transpo_degree)

    # Formula subject to change
    return external_weight * (transpo_stop_ratio * avg_transpo_degree) + internal_weight * num_intersections, transpo_stop_ratio,num_intersections, avg_transpo_degree


def compute_random_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        selected_node = random.choice(list(graph_copy.nodes()))
        graph_copy.remove_node(selected_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_targeted_failure_robustness(road_snapped_network_graph, num_removals):
    graph_copy = road_snapped_network_graph.copy() # Make a copy
    
    for i in range(num_removals):
        node_degrees = graph_copy.degree()
        # Iterate over the DegreeView object to find the maximum degree
        max_degree = max(degree for _, degree in node_degrees)
        max_degree_node = get_node_with_degree(node_degrees, max_degree)
        graph_copy.remove_node(max_degree_node)

    diameter, avg_path_length = compute_network_statistics(graph_copy)
    return compute_failure_robustness(graph_copy, diameter)

def compute_failure_robustness(road_snapped_network_graph, max_path_length):
    return float(max_path_length) / float(len(road_snapped_network_graph) - 1)

def compute_network_statistics(road_snapped_network_graph):
    path_lengths = get_path_lengths(road_snapped_network_graph) # Get the sum of all possible
    avg_path_length = np.mean(path_lengths)
    max_path_length = max(path_lengths)

    #network_size = len(path_lengths)
    #gcc = sorted(nx.connected_component_subgraphs(road_snapped_network_graph), key=len, reverse=True)
    #giant_component_fraction = float(float(gcc[0].order()) / float(network_size))
    #return max_path_length, avg_path_length, giant_component_fraction
    return max_path_length, avg_path_length

def get_node_with_degree(node_degrees, degree):
    # Iterate over the DegreeView object to find the node with the specified degree
    for node, _ in node_degrees:
        if _ == degree:
            return node
    return None  # Return None if no node with the specified degree is found

def get_path_lengths(snapped_road_network_graph):
    return [sum(nx.single_source_shortest_path_length(snapped_road_network_graph, n).values())
            for n in snapped_road_network_graph]
    
def get_network_coverage(network_graph, total_road_network_distance, graph_of_stops):
    
    travelled_distance = 0
    edge_set = [] # This will contain all the edges travelled
    path_set = []
    
    for node1, node2, edge_data in network_graph.edges(data=True):
        string_key1 = f"{node1}_{node2}"
        string_key2 = f"{node2}_{node1}"
        
        if string_key1 not in edge_set and string_key2 not in edge_set:
            edge_set.append(string_key1)
            edge_set.append(string_key2)
            path_travelled = edge_data['road_path']
            
            prev_stop = None
            for stop in path_travelled:
                if prev_stop == None:
                    prev_stop = stop
                    continue
                    
                key = f"{prev_stop}_{stop}"
                edge = graph_of_stops.get_edge_data(prev_stop, stop)
                data = list(edge.values())[0]
                edge_highway = data['highway']
                
                rows_with_lists['highway'].apply(check_list)
                if key not in path_set:
                    if isinstance(edge_highway, list) and (any(element in edge_highway for element in filter_options)) or edge_highway in filter_options:
                        travelled_distance += data['length']
                        path_set.append(key)
                        
                prev_stop = stop
            
    #print(travelled_distance, '/', TOTAL_NETWORK_ROAD_DISTANCE, '=', travelled_distance / TOTAL_NETWORK_ROAD_DISTANCE) 
    return travelled_distance / total_road_network_distance

# Get the roads whose highway types are part of the list
def check_list(lst):
    return any(x in filter_options for x in lst)

### Network Analysis Metrics

In [89]:
# Longest Route
def get_longest_route(network_routes):
    sorted_routes = sorted(network_routes, key=lambda x: x.length, reverse=True)
    longest_route = sorted_routes[0]
    
    stop_list = [] # Get all the stops
    for connection in longest_route.route:
        if connection[0] not in stop_list:
            stop_list.append(connection[0])
        if connection[-1] not in stop_list:
            stop_list.append(connection[-1])
    
    return longest_route, stop_list

# Shortest Route
def get_shortest_route(network_routes):
    sorted_routes = sorted(network_routes, key=lambda x: x.length, reverse=True)
    shortest_route = sorted_routes[-1]
    
    stop_list = [] # Get all the stops
    for connection in shortest_route.route:
        if connection[0] not in stop_list:
            stop_list.append(connection[0])
        if connection[-1] not in stop_list:
            stop_list.append(connection[-1])
    
    return shortest_route, stop_list

# Average Route Length
def get_average_route_length(network_routes):
    sum_length = sum([n.length for n in network_routes])
    average_length = sum_length / len(network_routes)
    return average_length

def get_longest_path_lengths(snapped_road_network_graph):
    path_lengths = []
    for node in snapped_road_network_graph.nodes():
        lengths = nx.single_source_dijkstra_path_length(snapped_road_network_graph, node, weight='distance')
        list_lengths = [n for target_node, n in lengths.items()]
        path_lengths.append(max(list_lengths))
    return path_lengths

#Network Diameter
def get_network_diameter(network_graph):
    path_lengths = get_longest_path_lengths(network_graph) # Get the sum of all possible
    max_path_length = max(path_lengths)
    return max_path_length

# def get_path_lengths(snapped_road_network_graph):
#     return [sum(nx.single_source_shortest_path_length(snapped_road_network_graph, n).values())
#             for n in snapped_road_network_graph]

# # Network Diameter
# def get_network_diameter(network_graph):
#     path_lengths = get_path_lengths(network_graph) # Get the sum of all possible
#     max_path_length = max(path_lengths)
#     return max_path_length

def average_degree(graph):
    degrees = [degree for _, degree in graph.degree()]
    avg_degree = sum(degrees) / len(degrees)
    return avg_degree

def average_number_of_stops_per_route(network_routes):
    num_stops = []
    for route in network_routes:
        seen = set()
        for connection in route.route:
            if connection[0] not in seen:
                seen.add(connection[0])
            
            if connection[-1] not in seen:
                seen.add(connection[-1])
                
        num_stops.append(len(seen))
            
    avg_stops = sum(num_stops) / len(num_stops)
    return avg_stops

In [90]:
# Node Centrality
def get_node_centrality(network_graph):
    degree_centrality = nx.degree_centrality(network_graph)
    betweenness_centrality = nx.betweenness_centrality(network_graph)
    closeness_centrality = nx.closeness_centrality(network_graph)
    
    return degree_centrality, betweenness_centrality, closeness_centrality

### Visualizations

In [91]:
def add_markers(routes, map):
    added = set()
    for route in routes:
        for stop in route:
            if stop.id not in added:
                #popup_text = f"Name: {stop.name}<br>Type: {stop.a_type}<br>Coordinates: {stop.getLat()}, {stop.getLong()}"
                lat = stop.lat
                long = stop.long
                
                if stop.isTranspo:
                    folium.Marker(location=[lat, long],icon=folium.Icon(color='green')).add_to(map)
                else:
                    folium.Marker(location=[lat, long],icon=folium.Icon(color='blue')).add_to(map)
                added.add(stop.id)

In [92]:
# This is to visualize the stops
def plot_stops_on_map(stops):
    # Iterate over the nodes in the network
    map_center = (14.599512, 120.984222)
    network_map = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
    for stop in stops:
        
        if stop.isTranspo:
            marker_color = 'blue'
        else:
            marker_color = 'red'
            
        folium.Marker(location=[stop.lat, stop.long], popup=f"Transportation: {stop.isTranspo}", icon=folium.Icon(color=marker_color)).add_to(network_map)
        
    return network_map

In [93]:
# Test Manila Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
for route in manila_route_stops:
    for stop in route:
        folium.Marker(location=[stop[0], stop[1]], popup=f"{stop[0]}, {stop[1]}", icon=folium.Icon(color='blue')).add_to(m)
        
m.save('test_manila.html')

In [94]:
# Test Makati Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
for route in makati_route_stops:
    for stop in route:
        folium.Marker(location=[stop[0], stop[1]], popup=f"{stop[0]}, {stop[1]}", icon=folium.Icon(color='blue')).add_to(m)
        
m.save('test_makati.html')

In [95]:
# Test Mandaluyong Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')
for route in mandaluyong_route_stops:
    for stop, name in route:
        folium.Marker(location=[stop[0], stop[1]], popup=f"{stop[0]}, {stop[1]}", icon=folium.Icon(color='blue')).add_to(m)
        
m.save('test_mandaluyong.html')

In [210]:
def improved_route_plot(graph, route, color, map, route_id):
    for connection in route.route:
        route_coords = [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in connection]
        
        if route.route.index(connection) == 0:
            lat = graph.nodes[connection[0]]['y']
            long = graph.nodes[connection[0]]['x']
            isTranspo = graph.nodes[connection[0]]['isTranspo']
            
            folium.CircleMarker(
            location=[lat, long],
            radius=6,  # Adjust the radius as needed
            popup=f"Transportation: {isTranspo}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
            ).add_to(map)
            
        lat = graph.nodes[connection[-1]]['y']
        long = graph.nodes[connection[-1]]['x']
        isTranspo = graph.nodes[connection[-1]]['isTranspo']
        
        folium.CircleMarker(
        location=[lat, long],
        radius=6,  # Adjust the radius as needed
        popup=f"Transportation: {isTranspo}\n{lat},{long}",
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7
        ).add_to(map)
        
        folium.PolyLine(route_coords, popup=f"{route_id}", color=color).add_to(map)
    
    return map

## Main

### Creating the Stops

In [97]:
# File Paths
pikl_filepath = "Original Jeepney Routes/Saved Networks/"
map_filepath = "Original Jeepney Routes/Maps/"

#### Manila Stops

In [34]:
# MANILA
# Remove all points that are out of bounds
tolerance = 0.0009
manila_route_stops_new = remove_if_out_of_bounds(MANILA_GRAPH, manila_route_stops, tolerance)

# Remove all routes that are less than 3 stops
temp_array = [] # For storing routes that are above 3 stops
for route in manila_route_stops_new:
    if len(route) > 3:
        temp_array.append(route)
manila_route_stops_new = temp_array

TypeError: 'float' object is not subscriptable

In [ ]:
# Getting all stops that are transpos (Stops that are near a transpo point are considered transpo stops)
max_distance_to_transpo_Manila = 30
# Get transpo stops
transpo_points_Manila, non_transpo_points_Manila, complete_route_list_Manila = check_transpo_stops(manila_route_stops_new, manila_points_gdf, manila_idx, max_distance_to_transpo)

In [ ]:
# Merge stops that are near each other
merged_list = transpo_points_Manila + non_transpo_points_Manila
stop_dict = merge_stops(merged_list)

route_list_Manila = copy.deepcopy(complete_route_list_Manila)
transpo_points_Manila, non_transpo_points_Manila = change_merged_stops(route_list_Manila, non_transpo_points_Manila, transpo_points_Manila, stop_dict)

In [ ]:
print(f"Number of non-transpo stops: {len(non_transpo_points_Manila)}")
print(f"Number of transpo stops: {len(transpo_points_Manila)}")
print(f"Ratio of transpo: {len(transpo_points_Manila) / (len(transpo_points_Manila) + len(non_transpo_points_Manila))}")

Number of non-transpo stops: 311
Number of transpo stops: 53
Ratio of transpo: 0.14560439560439561


In [ ]:
# MANILA STOPCANDIDATES (Turn stops into stopCandidates and plot them on the map)
graph_of_stops_Manila = copy.deepcopy(MANILA_GRAPH)
list_of_stops_Manila = []

place_stops_on_roads(transpo_points_Manila, non_transpo_points_Manila, graph_of_stops_Manila, list_of_stops_Manila)

# Visualize the stops
map = plot_stops_on_map(list_of_stops_Manila)
map.save("stop_map_manila.html")

In [ ]:
#Export stops to pickle
file_path = f'{pikl_filepath}stop_list_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(list_of_stops_Manila, f)
    
file_path = f'{pikl_filepath}stop_graph_Manila.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(graph_of_stops_Manila, f)

#### Makati

In [ ]:
# MAKATI
# Remove all points that are out of bounds
tolerance = 0.009
makati_route_stops_new = remove_if_out_of_bounds(MAKATI_GRAPH, makati_route_stops, tolerance)

# Remove all routes that are less than 3 stops
temp_array = [] # For storing routes that are above 3 stops
for route in makati_route_stops_new:
    if len(route) > 3:
        temp_array.append(route)
makati_route_stops_new = temp_array

In [ ]:
# Get transpo stops
max_distance_to_transpo_Makati = 50
transpo_points_Makati, non_transpo_points_Makati, complete_route_list_Makati = check_transpo_stops(makati_route_stops_new, makati_points_gdf, makati_idx, max_distance_to_transpo_Makati)

In [ ]:
# Merge stops in Makati that are near each other

merged_list = transpo_points_Makati + non_transpo_points_Makati
stop_dict = merge_stops(merged_list)

route_list_Makati = copy.deepcopy(complete_route_list_Makati)
transpo_points_Makati, non_transpo_points_Makati = change_merged_stops(route_list_Makati, non_transpo_points_Makati, transpo_points_Makati, stop_dict)

In [ ]:
print(f"Number of non-transpo stops: {len(non_transpo_points_Makati)}")
print(f"Number of transpo stops: {len(transpo_points_Makati)}")
print(f"Ratio of transpo: {len(transpo_points_Makati) / (len(transpo_points_Makati) + len(non_transpo_points_Makati))}")

Number of non-transpo stops: 134
Number of transpo stops: 8
Ratio of transpo: 0.056338028169014086


In [ ]:
# MAKATI STOPCANDIDATES
graph_of_stops_Makati = copy.deepcopy(MAKATI_GRAPH)
list_of_stops_Makati = []

place_stops_on_roads(transpo_points_Makati, non_transpo_points_Makati, graph_of_stops_Makati, list_of_stops_Makati)

# Visualize the stops
map = plot_stops_on_map(list_of_stops_Makati)
map.save("stop_map_makati.html")

In [ ]:
#Export stops to pickle
file_path = f'{pikl_filepath}stop_list_Makati.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(list_of_stops_Makati, f)
    
file_path = f'{pikl_filepath}stop_graph_Makati.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(graph_of_stops_Makati, f)

#### Mandaluyong

In [239]:
# MANDALUYONG
# Remove all points that are out of bounds
tolerance = 0.09
mandaluyong_route_stops_new = remove_if_out_of_bounds(MANDALUYONG_GRAPH, mandaluyong_route_stops, tolerance)

# Remove all routes that are less than 3 stops
temp_array = [] # For storing routes that are above 3 stops
for route in mandaluyong_route_stops_new:
    if len(route) > 3:
        temp_array.append(route)
mandaluyong_route_stops_new = temp_array

In [240]:
# Get transpo stops
max_distance_to_transpo_Mandaluyong = 50
transpo_points_Mandaluyong, non_transpo_points_Mandaluyong, complete_route_list_Mandaluyong = check_transpo_stops(mandaluyong_route_stops_new, mandaluyong_points_gdf, mandaluyong_idx, max_distance_to_transpo_Mandaluyong)

In [241]:
merged_list = transpo_points_Mandaluyong + non_transpo_points_Mandaluyong
stop_dict = merge_stops(merged_list)

route_list_Mandaluyong = copy.deepcopy(complete_route_list_Mandaluyong)
transpo_points_Mandaluyong, non_transpo_points_Mandaluyong = change_merged_stops(route_list_Mandaluyong, non_transpo_points_Mandaluyong, transpo_points_Mandaluyong, stop_dict)

In [242]:
print(f"Number of non-transpo stops: {len(non_transpo_points_Mandaluyong)}")
print(f"Number of transpo stops: {len(transpo_points_Mandaluyong)}")
print(f"Ratio of transpo: {len(transpo_points_Mandaluyong) / (len(transpo_points_Mandaluyong) + len(non_transpo_points_Mandaluyong))}")

Number of non-transpo stops: 60
Number of transpo stops: 10
Ratio of transpo: 0.14285714285714285


In [243]:
# Test Mandaluyong Map

# Create a map centered at the initial location
map_center = (14.599512, 120.984222) # TEMPORARY WILL ZOOM TO MANILA
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

count  = 0
for i, data in mandaluyong_points_gdf.iterrows():
    if data['amenity'] == 'transportation':
        folium.Marker(location=[data['y'], data['x']], popup=f"{data['name']}", icon=folium.Icon(color='blue')).add_to(m)
        count += 1
print(count)
        
m.save('transpo_mandaluyong.html')

48


In [244]:
# MANDALUYONG STOPCANDIDATES
graph_of_stops_Mandaluyong = copy.deepcopy(MANDALUYONG_GRAPH)
list_of_stops_Mandaluyong = []

place_stops_on_roads(transpo_points_Mandaluyong, non_transpo_points_Mandaluyong, graph_of_stops_Mandaluyong, list_of_stops_Mandaluyong)

# Visualize the stops
map = plot_stops_on_map(list_of_stops_Mandaluyong)
map.save("stop_map_mandaluyong.html")

In [245]:
#Export stops to pickle
file_path = f'{pikl_filepath}stop_list_Mandaluyong.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(list_of_stops_Mandaluyong, f)
    
file_path = f'{pikl_filepath}stop_graph_Mandaluyong.pkl'
with open(file_path, 'wb') as f:
    pickle.dump(graph_of_stops_Mandaluyong, f)

### Creating the routes

In [ ]:
# Manila
route_network_Manila, route_network_graph_Manila, reverse_route_network_Manila = generate_route_network(route_list_Manila, graph_of_stops_Manila)
Manila_Route_Network = networkObj(route_network_Manila, route_network_graph_Manila)

0 - 1
1 - 771
771 - 772
772 - 4
4 - 775
775 - 6
6 - 7
7 - 8
8 - 9
9 - 779
779 - 900
900 - 12

13 - 14
14 - 15
15 - 16
16 - 17
17 - 18

19 - 21
21 - 24
24 - 25
25 - 26
26 - 27
27 - 28
28 - 29
29 - 30
30 - 212
212 - 32
32 - 36

43 - 44
44 - 45
45 - 46
46 - 47
47 - 49
49 - 50
50 - 52
52 - 55
55 - 57
57 - 58

59 - 60
60 - 61
61 - 62
62 - 63
63 - 64
64 - 65
65 - 67
67 - 71
71 - 72

73 - 74
74 - 75
75 - 76
76 - 77
77 - 81
81 - 82
82 - 83
83 - 84
84 - 85
85 - 86
86 - 87

88 - 89
89 - 90
90 - 409
409 - 92

93 - 94
94 - 95
95 - 96
96 - 97
97 - 98
98 - 99

100 - 101
101 - 102
102 - 103
103 - 104
104 - 105
105 - 106
106 - 107
107 - 108
108 - 95
95 - 110
110 - 98
98 - 99
99 - 113
113 - 114

60 - 59
59 - 117
117 - 118
118 - 25
25 - 26
26 - 27
27 - 29
29 - 124
124 - 125
125 - 126
126 - 127
127 - 128
128 - 129
129 - 130
130 - 833
833 - 132
132 - 133
133 - 19
19 - 36

137 - 138
138 - 139
139 - 140
140 - 141
141 - 142
142 - 143
143 - 144
144 - 145

146 - 147
147 - 148
148 - 149
149 - 46
46 - 152

156 -

In [ ]:
# MAKATI ROUTES
route_network_Makati, route_network_graph_Makati, reverse_route_network_Makati = generate_route_network(route_list_Makati, graph_of_stops_Makati)
Makati_Route_Network = networkObj(route_network_Makati, route_network_graph_Makati)

0 - 1
1 - 2
2 - 3
3 - 5
5 - 6
6 - 7
7 - 8
8 - 9

10 - 11
11 - 12
12 - 13
13 - 14
14 - 15
15 - 16
16 - 17

18 - 3
3 - 20
20 - 21
21 - 22
22 - 23
23 - 24
24 - 25
25 - 26
26 - 27
27 - 29
29 - 31
31 - 32
32 - 33
33 - 34
34 - 35
35 - 36

27 - 39
39 - 6
6 - 41
41 - 5


58 - 59
59 - 60
60 - 62

63 - 64
64 - 65
65 - 66
66 - 67
67 - 69

70 - 8
8 - 72
72 - 7
7 - 74
74 - 5
5 - 17
17 - 77
77 - 79
79 - 80
80 - 81
81 - 82

0 - 84
84 - 77
77 - 91
91 - 17
17 - 5
5 - 95
95 - 7
7 - 99
99 - 72
72 - 8
8 - 102
102 - 9

105 - 27
27 - 107
107 - 108
108 - 109
109 - 110
110 - 111
111 - 112
112 - 21
21 - 114
114 - 115
115 - 116

117 - 118
118 - 119
119 - 120
120 - 65
65 - 122

62 - 124
124 - 107
107 - 108
108 - 128

129 - 130
130 - 131
131 - 132
132 - 133
133 - 134
134 - 135

135 - 134
134 - 133
133 - 139
139 - 140
140 - 141
141 - 142
142 - 143
143 - 144

9 - 8
8 - 72
72 - 7
7 - 6
6 - 5
5 - 3
3 - 154

155 - 35
35 - 251
251 - 161
161 - 64
64 - 65
65 - 166

118 - 169
169 - 170
170 - 172
172 - 119
119 - 133
133 - 

In [246]:
# MANDALUYONG
route_network_Mandaluyong, route_network_graph_Mandaluyong, reverse_route_network_Mandaluyong = generate_route_network(route_list_Mandaluyong, graph_of_stops_Mandaluyong)
Mandaluyong_Route_Network = networkObj(route_network_Mandaluyong, route_network_graph_Mandaluyong)

0 - 12
12 - 13
13 - 3
3 - 97
97 - 16
16 - 89
89 - 7
7 - 8
8 - 36

10 - 0
0 - 12
12 - 13
13 - 3
3 - 97
97 - 16
16 - 89
89 - 7
7 - 36

56 - 47
47 - 104
104 - 23
23 - 24
24 - 25
25 - 26
26 - 27
27 - 28
28 - 29
29 - 30
30 - 31
31 - 32

33 - 56
56 - 35
35 - 36
36 - 37
37 - 38
38 - 39
39 - 40
40 - 64
64 - 42
42 - 43
43 - 44
44 - 45
45 - 46

47 - 104
104 - 23
23 - 50
50 - 51
51 - 52
52 - 53
53 - 54
54 - 55

56 - 57
57 - 58
58 - 59
59 - 60
60 - 61
61 - 62
62 - 63
63 - 64
64 - 89
89 - 66
66 - 16
16 - 97
97 - 69
69 - 70
70 - 71
71 - 72
72 - 73
73 - 74
74 - 12
12 - 0
0 - 77

78 - 79
79 - 80
80 - 81
81 - 82
82 - 83
83 - 84
84 - 85
85 - 86
86 - 87
87 - 88
88 - 89
89 - 64
64 - 36
36 - 56

0 - 12
12 - 13
13 - 3
3 - 97
97 - 16
16 - 99
99 - 100
100 - 24
24 - 50
50 - 23
23 - 104
104 - 47
47 - 10


Real route
0 - 12
12 - 13
13 - 3
3 - 97
97 - 16
16 - 89
89 - 7
7 - 8
8 - 36

[[36, 8], [8, 7], [7, 89], [89, 16], [16, 97], [97, 3], [3, 13], [13, 12], [12, 0]]
10 - 0
0 - 12
12 - 13
13 - 3
3 - 97
97 - 16
16 -

In [207]:
colors = [
    "Red", "Green", "Blue", "Orange", "Purple", "Magenta", "Maroon",
    "Olive", "Teal", "Navy", "Aqua", "Fuchsia", "Coral", "Indigo",
    "Violet", "Brown", "DarkGreen", "DarkBlue", "DarkRed", "DarkCyan",
    "DarkMagenta", "DarkOrange", "DarkViolet", "DarkTurquoise", "DarkSlateGray",
    "SaddleBrown", "SeaGreen", "MidnightBlue", "ForestGreen", "Sienna",
    "SteelBlue", "RoyalBlue", "SlateBlue", "MediumSeaGreen", "MediumSlateBlue",
    "MediumPurple", "MediumTurquoise", "Crimson", "Goldenrod", "Chocolate",
    "FireBrick", "Peru", "RosyBrown", "IndianRed", "SlateGray", "DodgerBlue",
    "LightSeaGreen", "MediumAquamarine", "CadetBlue"
] 

In [ ]:
# MANILA
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

for route in Manila_Route_Network.routes:
    chosen_color = random.choice(colors)
    improved_route_plot(graph_of_stops_Manila, route, chosen_color, m, route.route_id)

m.save(f"{map_filepath}Manila Route Map.html")

In [ ]:
# MAKATI
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

for route in Makati_Route_Network.routes:
    chosen_color = random.choice(colors)
    improved_route_plot(graph_of_stops_Makati, route, chosen_color, m, route.route_id)

m.save(f"{map_filepath}Makati Route Map.html")

In [247]:
# MANDALUYONG
map_center = (14.599512, 120.984222)
m = folium.Map(location=map_center, zoom_start=10, tiles='openstreetmap')

for route in Mandaluyong_Route_Network.routes:
    chosen_color = random.choice(colors)
    improved_route_plot(graph_of_stops_Mandaluyong, route, chosen_color, m, route.route_id)

m.save(f"{map_filepath}Mandaluyong Route Map.html")

In [ ]:
# SEE ALL ROADS OF MANDALUYONG
m = folium.Map(location=(14.599512, 120.984222), zoom_start=9)

gdf_edges = ox.graph_to_gdfs(MANDALUYONG_GRAPH, nodes=False, edges=True)

# Add lines (edges) to the map
for i,edge in gdf_edges.iterrows():
    line_coords = list(edge['geometry'].coords)
    folium.PolyLine(locations=[(y, x) for x, y in line_coords], color='blue').add_to(m)

# Save the map to an HTML file
m.save('city_graph_map.html')

### Exporting Route Networks

In [251]:
export_networks(Manila_Route_Network, f'{pikl_filepath}Manila Route Network.pikl')

export_networks(Makati_Route_Network, f'{pikl_filepath}Makati Route Network.pikl')

export_networks(Mandaluyong_Route_Network, f'{pikl_filepath}Mandaluyong Route Network.pikl')

In [136]:
export_networks(Mandaluyong_Route_Network, f'{pikl_filepath}Mandaluyong Route Network.pikl')

### Fitness Function Computation

In [44]:
def get_num_stops(route_list):
    seen = set()
    for route in route_list:
        for stop in route:
            if stop.id not in seen:
                seen.add(stop.id)
                
    return len(seen)

In [45]:
num_failure_removal = 10
weight_random_failure = 0.1
weight_targeted_failure = 0.1
weight_connectivity = 0.2
weight_network_coverage = 0.6

In [ ]:
# Evaluate the fitness of each network in the population


#num_failure_removal_Manila = get_num_stops(route_list_Manila) * num_failure_removal_percentage
Manila_fitness_score, data_Manila = compute_fitness_score(Manila_Route_Network.graph, num_failure_removal, weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, TOTAL_NETWORK_ROAD_DISTANCE_MANILA, graph_of_stops_Manila)
Manila_Route_Network.fitness_score = Manila_fitness_score
print(f"Manila Route Network Score {Manila_Route_Network.fitness_score}")
print(f"Weighted Connectivity: {data_Manila['Connectivity']}")
print(f"- Transpo Stop Ratio: {data_Manila['Transpo Stop Ratio']}")
print(f"- Num Intersections: {data_Manila['Num Intersections']}")
print(f"- Average Transpo Stop Degree: {data_Manila['Average Transpo Degree']}")
print(f"Weighted Network Coverage: {data_Manila['Network Coverage']}")
print(f"Weighted Random Failure Score: {data_Manila['Random Failure Score']}")
print(f"Weighted Target Failure Score: {data_Manila['Target Failure Score']}")
print(f"Total Stops: {len(Manila_Route_Network.graph.nodes(data=True))}")
print(f"Diameter: {get_network_diameter(Manila_Route_Network.graph)}")
print()


num_failure_removal =int(get_num_stops(route_list_Makati) * 0.02)
Makati_fitness_score, data_Makati = compute_fitness_score(Makati_Route_Network.graph, num_failure_removal, weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, TOTAL_NETWORK_ROAD_DISTANCE_MAKATI, graph_of_stops_Makati)
Makati_Route_Network.fitness_score = Makati_fitness_score
print(f"Makati Route Network Score {Makati_Route_Network.fitness_score}")
print(f"Weighted Connectivity: {data_Makati['Connectivity']}")
print(f"- Transpo Stop Ratio: {data_Makati['Transpo Stop Ratio']}")
print(f"- Num Intersections: {data_Makati['Num Intersections']}")
print(f"- Average Transpo Stop Degree: {data_Makati['Average Transpo Degree']}")
print(f"Weighted Network Coverage: {data_Makati['Network Coverage']}")
print(f"Weighted Random Failure Score: {data_Makati['Random Failure Score']}")
print(f"Weighted Target Failure Score: {data_Makati['Target Failure Score']}")
print(f"Total Stops: {len(Makati_Route_Network.graph.nodes(data=True))}")
print(f"Diameter: {get_network_diameter(Makati_Route_Network.graph)}")
print()

num_failure_removal =int(get_num_stops(route_list_Mandaluyong) * 0.02)
Mandaluyong_fitness_score, data_Mandaluyong = compute_fitness_score(Mandaluyong_Route_Network.graph, num_failure_removal, weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, TOTAL_NETWORK_ROAD_DISTANCE_MANDALUYONG, graph_of_stops_Mandaluyong)
Mandaluyong_Route_Network.fitness_score = Mandaluyong_fitness_score
print(f"Mandaluyong Route Network Score {Mandaluyong_Route_Network.fitness_score}")
print(f"Weighted Connectivity: {data_Mandaluyong['Connectivity']}")
print(f"- Transpo Stop Ratio: {data_Mandaluyong['Transpo Stop Ratio']}")
print(f"- Num Intersections: {data_Mandaluyong['Num Intersections']}")
print(f"- Average Transpo Stop Degree: {data_Mandaluyong['Average Transpo Degree']}")
print(f"Weighted Network Coverage: {data_Mandaluyong['Network Coverage']}")
print(f"Weighted Random Failure Score: {data_Mandaluyong['Random Failure Score']}")
print(f"Weighted Target Failure Score: {data_Mandaluyong['Target Failure Score']}")
print(f"Diameter: {get_network_diameter(Mandaluyong_Route_Network.graph)}")
print(f"Total Stops: {len(Mandaluyong_Route_Network.graph.nodes(data=True))}")
print()
print()








Manila Route Network Score 13.5469606221781
Weighted Connectivity: 16.221813031161474
- Transpo Stop Ratio: 0.1388101983002833
- Num Intersections: 161
- Average Transpo Stop Degree: 8.775510204081632
Weighted Network Coverage: 0.23801308809265048
Weighted Random Failure Score: 1.8619883040935674
Weighted Target Failure Score: 1.0508771929824563
Total Stops: 353
Diameter: 6962

Makati Route Network Score 4.069111549781465
Weighted Connectivity: 5.6728571428571435
- Transpo Stop Ratio: 0.17142857142857143
- Num Intersections: 56
- Average Transpo Stop Degree: 4.25
Weighted Network Coverage: 0.27873615874913943
Weighted Random Failure Score: 0.9576642335766424
Weighted Target Failure Score: 0.9248175182481753
Total Stops: 140
Diameter: 1379

Mandaluyong Route Network Score 2.50942145292836
Weighted Connectivity: 3.6700000000000004
- Transpo Stop Ratio: 0.12
- Num Intersections: 36
- Average Transpo Stop Degree: 5.833333333333333
Weighted Network Coverage: 0.3054008343716584
Weighted Rand

In [65]:
#num_failure_removal = int(get_num_stops(route_list_Mandaluyong) * 0.05)
num_failure_removal = 2
Mandaluyong_fitness_score, data_Mandaluyong = compute_fitness_score(Mandaluyong_Route_Network.graph, num_failure_removal, weight_random_failure, weight_network_coverage, weight_targeted_failure, weight_connectivity, TOTAL_NETWORK_ROAD_DISTANCE_MANDALUYONG, graph_of_stops_Mandaluyong)
Mandaluyong_Route_Network.fitness_score = Mandaluyong_fitness_score
print(f"Mandaluyong Route Network Score {Mandaluyong_Route_Network.fitness_score}")
print(f"Weighted Connectivity: {data_Mandaluyong['Connectivity']}")
print(f"- Transpo Stop Ratio: {data_Mandaluyong['Transpo Stop Ratio']}")
print(f"- Num Intersections: {data_Mandaluyong['Num Intersections']}")
print(f"- Average Transpo Stop Degree: {data_Mandaluyong['Average Transpo Degree']}")
print(f"Weighted Network Coverage: {data_Mandaluyong['Network Coverage']}")
print(f"Weighted Random Failure Score: {data_Mandaluyong['Random Failure Score']}")
print(f"Weighted Target Failure Score: {data_Mandaluyong['Target Failure Score']}")
print(f"Diameter: {get_network_diameter(Mandaluyong_Route_Network.graph)}")
print(f"Total Stops: {len(Mandaluyong_Route_Network.graph.nodes(data=True))}")
print()
print()

Mandaluyong Route Network Score -0.52369110898364
Weighted Connectivity: 1.4771929824561405
- Transpo Stop Ratio: 0.14035087719298245
- Num Intersections: 14
- Average Transpo Stop Degree: 5.5
Weighted Network Coverage: 0.09541220485651603
Weighted Random Failure Score: 1.0462962962962965
Weighted Target Failure Score: 1.05
Diameter: 14.647345988246244
Total Stops: 57




### Network Metrics Result

In [ ]:
def print_city_statistics(city_name, avg_betweenness, avg_closeness, longest_route, long_stops, shortest_route, short_stops, avg_route_length, avg_degree, avg_stops):
    print(f"City: {city_name}")
    print(f"  Average Betweenness Centrality: {avg_betweenness:.4f}")
    print(f"  Average Closeness Centrality: {avg_closeness:.4f}")
    print(f"  Longest Route: {longest_route}")
    print(f"  Shortest Route: {shortest_route}")
    print(f"  Average Route Length: {avg_route_length:.4f}")
    print(f"  Average Degree: {avg_degree:.4f}")
    print(f"  Average Number of Stops per Route: {avg_stops:.4f}")
    print("")

In [ ]:
_, betweenness_centrality_Manila, closeness_centrality_Manila = get_node_centrality(Manila_Route_Network.graph)
avg_betweenness_centrality_Manila = sum(betweenness_centrality_Manila.values()) / len(betweenness_centrality_Manila)
avg_closeness_centrality_Manila = sum(closeness_centrality_Manila.values()) / len(closeness_centrality_Manila)
longest_route_Manila, long_route_stops_Manila = get_longest_route(Manila_Route_Network.routes)
shortest_route_Manila, short_route_stops_Manila = get_shortest_route(Manila_Route_Network.routes)
avg_route_length_Manila = get_average_route_length(Manila_Route_Network.routes)
avg_degree_Manila = average_degree(Manila_Route_Network.graph)
avg_number_stops_Manila = average_number_of_stops_per_route(Manila_Route_Network.routes)

print_city_statistics(
    "Manila",
    avg_betweenness_centrality_Manila,
    avg_closeness_centrality_Manila,
    longest_route_Manila.length,
    long_route_stops_Manila,
    shortest_route_Manila.length,
    short_route_stops_Manila,
    avg_route_length_Manila,
    avg_degree_Manila,
    avg_number_stops_Manila
)

_, betweenness_centrality_Makati, closeness_centrality_Makati = get_node_centrality(Makati_Route_Network.graph)
avg_betweenness_centrality_Makati = sum(betweenness_centrality_Makati.values()) / len(betweenness_centrality_Makati)
avg_closeness_centrality_Makati = sum(closeness_centrality_Makati.values()) / len(closeness_centrality_Makati)
longest_route_Makati, long_route_stops_Makati = get_longest_route(Makati_Route_Network.routes)
shortest_route_Makati, short_route_stops_Makati = get_shortest_route(Makati_Route_Network.routes)
avg_route_length_Makati = get_average_route_length(Makati_Route_Network.routes)
avg_degree_Makati = average_degree(Makati_Route_Network.graph)
avg_number_stops_Makati = average_number_of_stops_per_route(Makati_Route_Network.routes)

print_city_statistics(
    "Makati",
    avg_betweenness_centrality_Makati,
    avg_closeness_centrality_Makati,
    longest_route_Makati.length,
    long_route_stops_Makati,
    shortest_route_Makati.length,
    short_route_stops_Makati,
    avg_route_length_Makati,
    avg_degree_Makati,
    avg_number_stops_Makati
)


City: Manila
  Average Betweenness Centrality: 0.0179
  Average Closeness Centrality: 0.0523
  Longest Route: 34.551636813592566
  Shortest Route: 0.9661522150057373
  Average Route Length: 13.2012
  Average Degree: 9.0425
  Average Number of Stops per Route: 11.2308

City: Makati
  Average Betweenness Centrality: 0.0308
  Average Closeness Centrality: 0.1024
  Longest Route: 26.689512989004193
  Shortest Route: 3.1233370848189486
  Average Route Length: 12.3651
  Average Degree: 6.3143
  Average Number of Stops per Route: 9.5000

City: Mandaluyong
  Average Betweenness Centrality: 0.0461
  Average Closeness Centrality: 0.0620
  Longest Route: 111.9072678201812
  Shortest Route: 6.328009852733587
  Average Route Length: 46.4401
  Average Degree: 5.4000
  Average Number of Stops per Route: 12.2500

